In [2]:
import os
from vllm import LLM, SamplingParams

os.environ['CUDA_VISIBLE_DEVICES'] = '2'
#os.environ['HF_HOME'] = '/mnt/scratch-artemis/haausing/hf_cache'

nb = "7b"
cache_dir = f"/mnt/scratch-artemis/haausing/hf_cache"
model_name = f"codellama/CodeLlama-{nb}-Python-hf"
llm = LLM(
    model=model_name, 
    max_model_len=2048,
    download_dir = cache_dir,
    )

/mnt/data/haausing/miniconda3/envs/evalplus/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-02-12 21:30:57,181	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 02-12 21:30:57 llm_engine.py:72] Initializing an LLM engine with config: model='codellama/CodeLlama-7b-Python-hf', tokenizer='codellama/CodeLlama-7b-Python-hf', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=2048, download_dir='/mnt/scratch-artemis/haausing/hf_cache', load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, seed=0)
INFO 02-12 21:31:01 weight_utils.py:164] Using model weights format ['*.safetensors']
INFO 02-12 21:31:05 llm_engine.py:322] # GPU blocks: 3763, # CPU blocks: 512
INFO 02-12 21:31:07 model_runner.py:632] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 02-12 21:31:07 model_runner.py:636] CUDA graphs can take additional 1~3 GiB memory per GPU. If you a

In [4]:
from generate import construct_contract_prompt

In [5]:
from evalplus.data import get_mbpp_plus
dataset = get_mbpp_plus()

In [18]:
#load prompt
task_id = "Mbpp/2"
task = dataset[task_id]
contract_type = "none"
prompt = construct_contract_prompt(task["prompt"], contract_type, task["contract"])
#load generation
output_dir = "/mnt/scratch-artemis/haausing/code_reranking/evalplus_outputs/mbpp/code-llama-7b_temp_0.8"
changed_task_id = task_id.replace("/", "_")
output_file = f"{output_dir}/{changed_task_id}/0.py"
# load output_file as str
with open(output_file, "r") as f:
    generation = f.read()

In [19]:
vllm_outputs = llm.generate(
    [prompt, generation],
    SamplingParams(
        temperature=1,
        max_tokens=1,
        top_p=1.0,
        prompt_logprobs=0
    ),
    use_tqdm=False,
)

In [22]:
prompt_ids = vllm_outputs[0].prompt_token_ids
assert vllm_outputs[1].prompt_token_ids[:len(prompt_ids)] == prompt_ids


In [34]:
log_probs = vllm_outputs[1].prompt_logprobs[len(prompt_ids):]# logprobs for the generation
# extract values of each dict
log_probs = [list(d.values())[0] for d in log_probs]

In [35]:
log_probs

[-0.3335464596748352,
 -0.5658361911773682,
 -0.09111369401216507,
 -0.11316433548927307,
 -0.0036429488100111485,
 -0.02000465989112854,
 -0.012292811647057533,
 -1.9872897863388062,
 -0.2220921516418457,
 -0.22801125049591064,
 -0.00937500037252903,
 -0.00028713393840007484,
 -0.05111154913902283,
 -0.0031545422971248627,
 -0.09514015167951584,
 -1.0171672105789185,
 -0.3694011867046356,
 -0.027140766382217407,
 -0.03143935278058052,
 -0.006101434119045734,
 -0.25642383098602295,
 -0.0021571479737758636,
 -0.0011356578906998038,
 -0.0007798014557920396,
 -0.6346943974494934,
 -0.0002215855201939121,
 -0.0001867835089797154,
 -0.0013525871327146888,
 -0.0005881248507648706,
 -0.03968319296836853]